# Data Exploration

## Setup

In [1]:
# Import libraries
import os
import pandas as pd
import requests

# Import API keys
from alyssa_config import spoonacular_key, rapidapi_key
# from lakna_config import spoonacular_key

In [2]:
# Check number of items in the Resources subdirectories, to automate file naming.
def recipe_folder():
    count = len(os.listdir('Resources/01_recipe_IDs'))-1
    return(count)

def raw_folder():
    count = len(os.listdir('Resources/02_raw_data'))-1
    return(count)

def simple_folder():
    count = len(os.listdir('Resources/03_simplified_data'))-1
    return(count)

In [3]:
# Request user input for 'initial_recipes' index of interest.
def check_index():
    try:
        select_folder = input(f"Select file type (recipe, raw, simple): ")
        if (select_folder == "recipe"):
            get_count = recipe_folder()
        elif (select_folder == "raw"):
            get_count = raw_folder()
        elif (select_folder == "simple"):
            get_count = simple_folder()
    except ValueError as e:
        print(f"{e}: Selected file type unknown.")

    try:
        # index = int(input(f"Select file number (0-{recipe_folder()-1}): "))
        index = int(input(f"Select file number (0-{get_count-1}): "))
        return index
    except ValueError as e:
        print(f"{e}: Integer required.")

## Initial Data Exploration - FOOD.COM Dataset
__Goal:__ Understand dataset format and identify attributes.

__PROCESS__
1. Extract the ratings from `RAW_interactions.csv` first.
2. Merge with the `RAW_recipes.csv` then investigate the dataset.

__OBSERVATIONS__
- Nutrition headings: calories (#), total fat (PDV), sugar (PDV) , sodium (PDV) , protein (PDV) , saturated fat (PDV), carbohydrates (PDV)
- Will need to convert the nutrition PDV to actual values later by defining a caloric diet level.

In [4]:
# Import the FOOD.COM datasets as DataFrames.
food_df = pd.read_csv('Resources/RAW_recipes.csv')
interactions_df = pd.read_csv('Resources/RAW_interactions.csv')

In [5]:
# Get the DataFrame dimensions
interactions_shape = interactions_df.shape
print(f"interactions_df: {interactions_shape}")

# Display the DataFrame
interactions_df.head()

interactions_df: (1132367, 5)


,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."


In [6]:
# Get the DataFrame dimensions and columns
food_shape = food_df.shape
print(f"food_df: {food_shape}")
print(food_df.columns)

# Display the DataFrame
food_df.head()

food_df: (231637, 12)
Index(['name', 'id', 'minutes', 'contributor_id', 'submitted', 'tags',
       'nutrition', 'n_steps', 'steps', 'description', 'ingredients',
       'n_ingredients'],
      dtype='object')


,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


### Calculate the Average Recipe Rating

In [7]:
# Check how many recipes have ratings
unique_ratings = len(interactions_df['recipe_id'].unique())
print(f"{unique_ratings} out of {food_shape[0]} have ratings.")

# Check how many recipes have a '0' rating
zero_rating = interactions_df.loc[interactions_df['rating'] == 0]['recipe_id'].unique()
print(f"There are {len(zero_rating)} recipes with a '0' rating.")

# Drop the rows with a '0' rating
nonzero_df = interactions_df.loc[interactions_df['rating'] != 0]
print(f"nonzero_df: {nonzero_df.shape}")

231637 out of 231637 have ratings.
There are 35114 recipes with a '0' rating.
nonzero_df: (1071520, 5)


In [8]:
# Create a DataFrame with the average ratings per recipe ID
food_ratings = nonzero_df.groupby('recipe_id')['rating'].mean().reset_index()

# Display the DataFrame
food_ratings.head()

,recipe_id,rating
0,38,4.250000
1,39,3.000000
2,40,4.333333
3,41,4.500000
4,43,1.000000


In [9]:
# Rename the recipe ID column for merging with food_df
food_ratings = food_ratings.rename(columns={'recipe_id': 'id'})

# Merge the datasets and display updated DataFrame
merged_food = pd.merge(food_df, food_ratings, on='id')

# Display the DataFrame
merged_shape = merged_food.shape
print(f"merged_food: {merged_shape}")
merged_food.head()

merged_food: (226590, 13)


,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,rating
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,5.000000
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,4.666667
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,4.000000
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,4.500000
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,5.000000


### Parse the Nutrition Values

In [10]:
# Identify how to split the nutrition string and convert to float
test_string = merged_food['nutrition']

# Remove the square brackets
test_string = test_string[0].strip("[]")

# Split the string to a list
test_string = test_string.split(", ")

# Cast values to float
test_string = [float(value) for value in test_string]
test_string

[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]

In [11]:
# Parse each value in the `nutrition` column
# NOTE: calories (#), total fat (PDV), sugar (PDV), sodium (PDV) , protein (PDV), saturated fat, carbohydrates (PDV)
for df_idx, row in merged_food.iterrows():
    # Strip and split the string to a list
    values_list = row['nutrition'].strip("[]").split(", ")

    # Allocate each nutritional value to the correct column
    for idx, value in enumerate(values_list):
        if (idx == 0):
            merged_food.loc[df_idx, 'Calories'] = float(value)
        elif (idx == 1):
            merged_food.loc[df_idx, 'Total Fat (PDV)'] = float(value)
        elif (idx == 2):
            merged_food.loc[df_idx, 'Sugar (PDV)'] = float(value)
        elif (idx == 3):
            merged_food.loc[df_idx, 'Sodium (PDV)'] = float(value)
        elif (idx == 4):
            merged_food.loc[df_idx, 'Protein (PDV)'] = float(value)
        elif (idx == 5):
            merged_food.loc[df_idx, 'Saturated Fat (PDV)'] = float(value)
        elif (idx == 6):
            merged_food.loc[df_idx, 'Carbohydrates (PDV)'] = float(value)

# Display the DataFrame
merged_food.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,rating,Calories,Total Fat (PDV),Sugar (PDV),Sodium (PDV),Protein (PDV),Saturated Fat (PDV),Carbohydrates (PDV)
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,5.000000,51.5,0.0,13.0,0.0,2.0,0.0,4.0
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,4.666667,173.4,18.0,0.0,17.0,22.0,35.0,1.0
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,4.000000,269.8,22.0,32.0,48.0,39.0,27.0,5.0
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,4.500000,368.1,17.0,10.0,2.0,14.0,8.0,20.0
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,5.000000,352.9,1.0,337.0,23.0,3.0,0.0,28.0


### Identify Duplicate Recipes

In [12]:
# Determine whether there are duplicate recipes by ID
dup_id = len(merged_food['id'].unique())
print(f"Unique recipe IDs: {dup_id}, max number of recipes? {merged_shape[0] == dup_id}")

# Determine whether there are duplicate recipes by name
dup_name = len(merged_food['name'].unique())
print(f"Unique recipe names: {dup_name}, max number of recipes? {merged_shape[0] == dup_name}")

# Get the duplicate names - these should be okay
duplicate_names = merged_food.loc[food_df.duplicated(['name'])]
duplicate_names.head()

Unique recipe IDs: 226590, max number of recipes? True
Unique recipe names: 225190, max number of recipes? False


,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,rating,Calories,Total Fat (PDV),Sugar (PDV),Sodium (PDV),Protein (PDV),Saturated Fat (PDV),Carbohydrates (PDV)
600,sesame noodles with peanut sauce,97968,30,14823,2004-08-17,"['30-minutes-or-less', 'time-to-make', 'course...","[472.5, 18.0, 60.0, 56.0, 36.0, 12.0, 24.0]",7,"['mince the garlic in a food processor , stopp...",delicious cold dish for a hot day.,"['garlic', 'natural-style peanut butter', 'soy...",8,4.666667,472.5,18.0,60.0,56.0,36.0,12.0,24.0
846,100 authentic vietnamese dipping sauce nuoc ...,289509,15,219695,2008-03-01,"['lactose', '15-minutes-or-less', 'time-to-mak...","[235.3, 0.0, 207.0, 109.0, 5.0, 0.0, 19.0]",5,"['in a large bowl , combine lime juice , garli...",i got this recipe from my mom (born and raised...,"['garlic', 'limes, juice of', 'white vinegar',...",7,5.000000,235.3,0.0,207.0,109.0,5.0,0.0,19.0
1314,30 minute bacon spaghetti for 2,157534,30,83093,2006-02-24,"['30-minutes-or-less', 'time-to-make', 'course...","[1221.2, 100.0, 102.0, 86.0, 70.0, 112.0, 40.0]",12,"['brown bacon', 'pour off all but 1 tablespoon...",super fast & easy! feel free to substitute an...,"['bacon', 'onion', 'red pepper', 'parsley', 'p...",8,4.500000,1221.2,100.0,102.0,86.0,70.0,112.0,40.0
1315,30 minute baked potato,155496,25,252156,2006-02-10,"['lactose', '30-minutes-or-less', 'time-to-mak...","[164.0, 0.0, 6.0, 0.0, 8.0, 0.0, 12.0]",6,['stab the potato with a fork several times an...,"cut the time, but not the lovely oven cooked t...","['potato', 'olive oil']",2,5.000000,164.0,0.0,6.0,0.0,8.0,0.0,12.0
1335,30 minute shepherd s pie,145473,40,63858,2005-11-19,"['60-minutes-or-less', 'time-to-make', 'course...","[847.2, 78.0, 20.0, 20.0, 91.0, 114.0, 17.0]",21,"['boil potatoes in salted water until tender ,...",from rachael ray. i love this recipe it's so ...,"['potatoes', 'sour cream', 'egg yolk', 'cream'...",15,4.500000,847.2,78.0,20.0,20.0,91.0,114.0,17.0


### Parse the Tags

In [13]:
# Identify how to isolate each tag
tag_string = merged_food['tags'][0].strip("[]")
tag_string = tag_string.split(', ')
tag_string[0].strip("'")

# Get a list of unique tags
unique_tags = []
for string in merged_food['tags']:
    tag_list = string.strip("[]").split(', ')
    for word_idx in range(len(tag_list)):
        tag = tag_list[word_idx].strip("'")
        if tag not in unique_tags:
            unique_tags.append(tag)

# Display the tags alphabetically
sorted(unique_tags)

['',
 '1-day-or-more',
 '15-minutes-or-less',
 '3-steps-or-less',
 '30-minutes-or-less',
 '4-hours-or-less',
 '5-ingredients-or-less',
 '60-minutes-or-less',
 'Throw the ultimate fiesta with this sopaipillas recipe from Food.com.',
 'a1-sauce',
 'african',
 'american',
 'amish-mennonite',
 'angolan',
 'appetizers',
 'apples',
 'april-fools-day',
 'argentine',
 'artichoke',
 'asian',
 'asparagus',
 'australian',
 'austrian',
 'avocado',
 'bacon',
 'baja',
 'baked-beans',
 'baking',
 'bananas',
 'bar-cookies',
 'barbecue',
 'bass',
 'bean-soup',
 'beans',
 'beans-side-dishes',
 'bear',
 'beef',
 'beef-barley-soup',
 'beef-kidney',
 'beef-liver',
 'beef-organ-meats',
 'beef-ribs',
 'beef-sauces',
 'beef-sausage',
 'beginner-cook',
 'beijing',
 'belgian',
 'berries',
 'beverages',
 'birthday',
 'biscotti',
 'bisques-cream-soups',
 'black-bean-soup',
 'black-beans',
 'blueberries',
 'bok-choys',
 'brazilian',
 'bread-machine',
 'bread-pudding',
 'breads',
 'breakfast',
 'breakfast-casserole

#### Extract `Meal Types` from the Tags

In [14]:
# Check the tags for meal types
meal_types = ["breakfast", "lunch", "dinner"]
for meal in meal_types:
    if meal in unique_tags:
        print(f"{meal} is a tag.")
    else:
        print(f"{meal} is not a tag.")

# Look for a similar tag to "dinner"
found = False
for tag in unique_tags:
    if (tag[:6] == "dinner"):
        print(f"{tag} is a match.")
        found = True
if (found is False):
    print("No matches found.")

breakfast is a tag.
lunch is a tag.
dinner is not a tag.
dinner-party is a match.


In [16]:
# Count how many rows contain each meal type
updated_meals = ["breakfast", "lunch", "dinner-party"]

break_count = 0
lunch_count = 0
dinner_count = 0
multiple = 0
total = 0

break_list = []
lunch_list = []
dinner_list = []

for idx, string in merged_food.iterrows():
    # Strip and split the string to a list
    tag_list = string['tags'].strip("[]").split(', ')

    # Get each tag from the list and categorise
    for word_idx in range(len(tag_list)):
        # Score tracks mutual exclusivity of the meal types
        score = 0

        # Check the tag
        tag = tag_list[word_idx].strip("'")
        if (tag in updated_meals):
            total += 1
            score = 0
            if (tag == "breakfast"):
                break_list.append(idx)
                break_count += 1
                score += 1
            if (tag == "lunch"):
                lunch_list.append(idx)
                lunch_count += 1
                score += 1
            if (tag == "dinner-party"):
                dinner_list.append(idx)
                dinner_count += 1
                score += 1
        if (score > 1):
            multiple += 1
combined_list = break_count + lunch_count + dinner_count
overlap = not(total == combined_list)
if (overlap is True):
    conclusion = "Recipes exist in more than one meal type."
elif (overlap is False):
    conclusion = "Recipes belong to only one meal type."

# Print findings
print(f"Total recipes with at least one tag: {total}")
print(32*'*')
print(f"Breakfast: {break_count}")
print(f"Lunch: {lunch_count}")
print(f"Dinner: {dinner_count}")
print(32*'*')
print(f"Multiple: {multiple}")
print(f"Checksum: {combined_list}")
print(f"Overlap? {overlap}. {conclusion}")

Total recipes with at least one tag: 73641
********************************
Breakfast: 13375
Lunch: 23392
Dinner: 36874
********************************
Multiple: 0
Checksum: 73641
Overlap? False. Recipes belong to only one meal type.


In [ ]:
test_df = merged_food.iloc[break_list]
subset = test_df[['rating', 'n_ingredients']]
subset.plot(kind="scatter", x='rating', y='n_ingredients')

In [ ]:
test2_df = merged_food.iloc[combined_list]
print(test2_df.shape)
test2_df.plot(kind="scatter", x="n_steps", y="Calories")

In [ ]:
test2_df

#### Extract `Cuisines` from the Tags

## Initial Data Exploration - SPOONACULAR API
__Goal:__ Test the API, check returned format and attributes.

__Process__
1. Get recipe IDs, these are needed to pass to other methods (i.e. `Recipe Information`).
2. Get recipe metadata using `Recipe Information`.
3. Save raw response to a CSV to minimise API request repetition.

#### Parse API Response

### Complex Search and Recipe Information
- Complex Search: [https://spoonacular.com/food-api/docs#Search-Recipes-Complex](https://spoonacular.com/food-api/docs#Search-Recipes-Complex)
- Recipe Information: [https://spoonacular.com/food-api/docs#Get-Recipe-Information](https://spoonacular.com/food-api/docs#Get-Recipe-Information)

__OBSERVATIONS__
- `Complex Search` returns the same result every time, need to try `Random Recipes`.

In [ ]:
# API REQUEST - Complex Search.
recipe_url = "https://api.spoonacular.com/recipes/complexSearch"

# Set request parameters
recipe_params = {
    'apiKey': spoonacular_key,
    'number': 100}

# Request a list of recipes.
recipe_response = requests.get(recipe_url, recipe_params).json()

# Output response to a csv
recipes_df = pd.DataFrame(recipe_response['results'])
recipes_df.to_csv(f'Resources/01_recipe_IDs/initial_recipes_0.csv', index=False)

In [ ]:
# Import and display complex search results
recipes_df = pd.read_csv(f'Resources/01_recipe_IDs/initial_recipes_0.csv')
recipes_df.head()

In [ ]:
# Check the number of returned recipes
print(f'Number of recipes returned: {len(recipes_df)}')

In [ ]:
# API REQUEST - Recipe Information
recipe_info = []
for id in recipes_df['id']:
    info_url = f"https://api.spoonacular.com/recipes/{id}/information?"
    info_params = {
        'apiKey': spoonacular_key,
        'includeNutrition': "true"}

    info_response = requests.get(info_url, info_params).json()
    recipe_info.append(info_response)

info_df = pd.DataFrame(recipe_info)
info_df.to_csv(f'Resources/02_raw_data/info_master_0.csv')

In [ ]:
# While the API response is accessible, identify attributes of interest
recipe_info[0].keys()

# Generic metadata
recipe_info[0]['id']
recipe_info[0]['title']
recipe_info[0]['aggregateLikes'] # will need to convert this to a 5-point scale
recipe_info[0]['extendedIngredients'] # can use len() to get the number of ingredients
recipe_info[0]['weightWatcherSmartPoints'] # research the equation for this
recipe_info[0]['cuisines'] # identify recipes with non-empty results
recipe_info[0]['dishTypes'] # will need to select a simpler subset

# Nutritional values - amount, unit, percent of daily needs
recipe_info[0]['nutrition']['nutrients'][0] # Calories
recipe_info[0]['nutrition']['nutrients'][2] # Saturated Fat
recipe_info[0]['nutrition']['nutrients'][5] # Sugar
recipe_info[0]['nutrition']['nutrients'][7] # Sodium
recipe_info[0]['nutrition']['nutrients'][8] # Protein

In [ ]:
# Import and display recipe information results
info_df = pd.read_csv(f'Resources/02_raw_data/info_master_0.csv')

# Get the list of column names
info_df.columns

# Display the dataframe
info_df.head()

### Random Recipes and Recipe Information
- Random Recipes: [https://spoonacular.com/food-api/docs#Get-Random-Recipes](https://spoonacular.com/food-api/docs#Get-Random-Recipes)
- Recipe Information: [https://spoonacular.com/food-api/docs#Get-Recipe-Information](https://spoonacular.com/food-api/docs#Get-Recipe-Information)

__OBSERVATIONS__
- Once converted to a dataframe, the data is not as accessible as navigating the API response, will need to create a simplified dataframe while the API response is available.
- Although this has metadata, it still does not have `Nutrition` information, will still need to run the `Recipe Information` request. Need to identify a cost-effective API request methodology.

In [ ]:
# API REQUEST - Random Recipe
random_url = "https://api.spoonacular.com/recipes/random?"

# Set request parameters
random_params = {
    'apiKey': spoonacular_key,
    'number': 100}

# Request a list of random recipe IDs.
random_response = requests.get(random_url, random_params).json()

# Output response to a csv
random_df = pd.DataFrame(random_response['recipes'])
random_df.to_csv(f'Resources/01_recipe_IDs/initial_recipes_4.csv', index=False)

In [ ]:
# Import and display random recipes results
random_df = pd.read_csv(f'Resources/01_recipe_IDs/initial_recipes_4.csv')
random_df.head()

In [ ]:
# Get the list of column name
random_df.columns

In [ ]:
# API REQUEST - Recipe Information
recipe_info = []
for id in random_df['id']:
    info_url = f"https://api.spoonacular.com/recipes/{id}/information?"
    info_params = {
        'apiKey': spoonacular_key,
        'includeNutrition': "true"}

    info_response = requests.get(info_url, info_params).json()
    recipe_info.append(info_response)

info_df = pd.DataFrame(recipe_info)
info_df.to_csv(f'Resources/02_raw_data/info_master_4.csv')

In [ ]:
# Display the dataframe
info_df.head()

### Parse API Response

__OBSERVATIONS__
- Ensure all relevant columns are accounted for before bulk requesting.
- Will need to account for different units.

In [ ]:
# Metadata
recipe_id = []
recipe_name = []
recipe_likes = []
meal_type = []
cuisines = []
num_ingredients = []
ww_points = []

# Calories
calories_amt = []
calories_unit = []
calories_pct = []

# Saturated Fat
satfat_amt = []
satfat_unit = []
satfat_pct = []

# Sugar
sugar_amt = []
sugar_unit = []
sugar_pct = []

# Sodium
sodium_amt = []
sodium_unit = []
sodium_pct = []

# Protein
protein_amt = []
protein_unit = []
protein_pct = []

for idx, row in enumerate(recipe_info):
    try:
        recipe_id.append(recipe_info[idx]['id'])
        recipe_name.append(recipe_info[idx]['title'])
        recipe_likes.append(recipe_info[idx]['aggregateLikes'])
        meal_type.append(recipe_info[idx]['dishTypes'])
        cuisines.append(recipe_info[idx]['cuisines'])
        num_ingredients.append(len(recipe_info[idx]['extendedIngredients']))
        ww_points.append(recipe_info[idx]['weightWatcherSmartPoints'])
        
        calories_amt.append(recipe_info[idx]['nutrition']['nutrients'][0]['amount'])
        calories_unit.append(recipe_info[idx]['nutrition']['nutrients'][0]['unit'])
        calories_pct.append(recipe_info[idx]['nutrition']['nutrients'][0]['percentOfDailyNeeds'])
        
        satfat_amt.append(recipe_info[idx]['nutrition']['nutrients'][2]['amount'])
        satfat_unit.append(recipe_info[idx]['nutrition']['nutrients'][2]['unit'])
        satfat_pct.append(recipe_info[idx]['nutrition']['nutrients'][2]['percentOfDailyNeeds'])
        
        sugar_amt.append(recipe_info[idx]['nutrition']['nutrients'][5]['amount'])
        sugar_unit.append(recipe_info[idx]['nutrition']['nutrients'][5]['unit'])
        sugar_pct.append(recipe_info[idx]['nutrition']['nutrients'][5]['percentOfDailyNeeds'])
        
        sodium_amt.append(recipe_info[idx]['nutrition']['nutrients'][7]['amount'])
        sodium_unit.append(recipe_info[idx]['nutrition']['nutrients'][7]['unit'])
        sodium_pct.append(recipe_info[idx]['nutrition']['nutrients'][7]['percentOfDailyNeeds'])
        
        protein_amt.append(recipe_info[idx]['nutrition']['nutrients'][8]['amount'])
        protein_unit.append(recipe_info[idx]['nutrition']['nutrients'][8]['unit'])
        protein_pct.append(recipe_info[idx]['nutrition']['nutrients'][8]['percentOfDailyNeeds'])
    
    except:
        print(idx)

spoonacular_df = pd.DataFrame({
    'ID': recipe_id,
    'Name': recipe_name,
    'Likes': recipe_likes,
    'Meal Type': meal_type,
    'Cuisines': cuisines,
    'N_ingredients': num_ingredients,
    'WW Smart Points': ww_points,
    'Calories (Amount)': calories_amt,
    'Calories (Unit)': calories_unit,
    'Calories (% of Daily Needs)': calories_pct,
    'Saturated Fat (Amount)': satfat_amt,
    'Saturated Fat (Unit)': satfat_unit,
    'Saturated Fat (% of Daily Needs)': satfat_pct,
    'Sugar (Amount)': sugar_amt,
    'Sugar (Unit)': sugar_unit,
    'Sugar (% of Daily Needs)': sugar_pct,
    'Sodium (Amount)': sodium_amt,
    'Sodium (Unit)': sodium_unit,
    'Sodium (% of Daily Needs)': sodium_pct,
    'Protein (Amount)': protein_amt,
    'Protein (Unit)': protein_unit,
    'Protein (% of Daily Needs)': protein_pct})

spoonacular_df.to_csv(f"Resources/03_simplified_data/initial_nutrition_4.csv", index=False)

In [ ]:
# Check the units columns
units_columns = ['Calories (Unit)', 'Saturated Fat (Unit)', 'Sugar (Unit)', 'Sodium (Unit)', 'Protein (Unit)']
for col in units_columns:
    check = spoonacular_df['Protein (Unit)'].unique()
    print(f"Unit check: {col} {check}")

### Random Recipes and Nutrition by ID

In [ ]:
# API REQUEST - Random Recipe
random_url = "https://api.spoonacular.com/recipes/random?"

# Set request parameters
random_params = {
    'apiKey': spoonacular_key,
    'number': 100}

# Request a list of random recipe IDs.
random_response = requests.get(random_url, random_params).json()

# Output response to a csv
random_df = pd.DataFrame(random_response['recipes'])
random_df.to_csv('Resources/01_recipe_IDs/initial_recipes_5.csv', index=False)

In [ ]:
# API REQUEST - Nutrition by ID
nutrition_info = []
for id in random_df['id']:
    nutrition_url = f"https://api.spoonacular.com/recipes/{id}/nutritionWidget.json"
    nutrition_params = {'apiKey': spoonacular_key}

    nutrition_response = requests.get(nutrition_url, nutrition_params).json()
    nutrition_info.append(nutrition_response)

nutrition_df = pd.DataFrame(nutrition_info)
nutrition_df.to_csv('Resources/02_raw_data/info_master_5.csv')

In [ ]:
metadata.columns

In [ ]:
nutrition_info[0]['carbs']

In [ ]:
# Parse results
metadata = pd.read_csv('Resources/01_recipe_IDs/initial_recipes_5.csv')
metadata

# Metadata
num_ingredients = []
carbs = []
# for row in metadata['extendedIngredients']:
#     print(row)

# Calories
calories_amt = []
calories_unit = []
calories_pct = []

# Saturated Fat
satfat_amt = []
satfat_unit = []
satfat_pct = []

# Sugar
sugar_amt = []
sugar_unit = []
sugar_pct = []

# Sodium
sodium_amt = []
sodium_unit = []
sodium_pct = []

# Protein
protein_amt = []
protein_unit = []
protein_pct = []

for idx, row in enumerate(nutrition_info):
    try:
        # recipe_id.append(nutrition_info[idx]['id'])
        # recipe_name.append(nutrition_info[idx]['title'])
        # recipe_likes.append(nutrition_info[idx]['aggregateLikes'])
        # meal_type.append(nutrition_info[idx]['dishTypes'])
        # cuisines.append(nutrition_info[idx]['cuisines'])
        # num_ingredients.append(len(recipe_info[idx]['extendedIngredients']))
        # ww_points.append(recipe_info[idx]['weightWatcherSmartPoints'])

        num_ingredients.append(len(nutrition_info[idx]['ingredients']))
        carbs.append(nutrition_info[idx]['carbs'])
        calories_amt.append(nutrition_info[idx]['nutrients'][0]['amount'])
        calories_unit.append(nutrition_info[idx]['nutrients'][0]['unit'])
        calories_pct.append(nutrition_info[idx]['nutrients'][0]['percentOfDailyNeeds'])
        
        satfat_amt.append(nutrition_info[idx]['nutrients'][2]['amount'])
        satfat_unit.append(nutrition_info[idx]['nutrients'][2]['unit'])
        satfat_pct.append(nutrition_info[idx]['nutrients'][2]['percentOfDailyNeeds'])
        
        sugar_amt.append(nutrition_info[idx]['nutrients'][5]['amount'])
        sugar_unit.append(nutrition_info[idx]['nutrients'][5]['unit'])
        sugar_pct.append(nutrition_info[idx]['nutrients'][5]['percentOfDailyNeeds'])
        
        sodium_amt.append(nutrition_info[idx]['nutrients'][7]['amount'])
        sodium_unit.append(nutrition_info[idx]['nutrients'][7]['unit'])
        sodium_pct.append(nutrition_info[idx]['nutrients'][7]['percentOfDailyNeeds'])
        
        protein_amt.append(nutrition_info[idx]['nutrients'][8]['amount'])
        protein_unit.append(nutrition_info[idx]['nutrients'][8]['unit'])
        protein_pct.append(nutrition_info[idx]['nutrients'][8]['percentOfDailyNeeds'])
    
    except:
        print(idx)

spoonacular_df = pd.DataFrame({
    'ID': metadata['id'],
    'Name': metadata['title'],
    'Likes': metadata['aggregateLikes'],
    'Meal Type': metadata['dishTypes'],
    'Cuisines': metadata['cuisines'],
    'N_ingredients': num_ingredients,
    'WW Smart Points': metadata['weightWatcherSmartPoints'],
    'Calories (Amount)': calories_amt,
    'Calories (Unit)': calories_unit,
    'Calories (% of Daily Needs)': calories_pct,
    'Saturated Fat (Amount)': satfat_amt,
    'Saturated Fat (Unit)': satfat_unit,
    'Saturated Fat (% of Daily Needs)': satfat_pct,
    'Sugar (Amount)': sugar_amt,
    'Sugar (Unit)': sugar_unit,
    'Sugar (% of Daily Needs)': sugar_pct,
    'Sodium (Amount)': sodium_amt,
    'Sodium (Unit)': sodium_unit,
    'Sodium (% of Daily Needs)': sodium_pct,
    'Protein (Amount)': protein_amt,
    'Protein (Unit)': protein_unit,
    'Protein (% of Daily Needs)': protein_pct,
    'Carbohydrates': carbs})

spoonacular_df.to_csv(f"Resources/03_simplified_data/initial_nutrition_5.csv", index=False)

# Data Collection
- API request cost comparison:
    1. `Random Recipes` + `Recipe Information` = 2.1 points per request
    2. `Random Recipes` + `Nutrition` = 2 points per request

In [ ]:
# API REQUEST - Random Recipe
random_url = "https://api.spoonacular.com/recipes/random?"

# Set request parameters
random_params = {
    'apiKey': spoonacular_key,
    'number': 100}

# Request a list of random recipe IDs.
random_response = requests.get(random_url, random_params).json()

# Output response to a csv
random_df = pd.DataFrame(random_response['recipes'])
random_df.to_csv(f'Resources/01_recipe_IDs/initial_recipes_{recipe_folder()}.csv', index=False)

In [ ]:
# API REQUEST - Nutrition by ID
nutrition_info = []
for id in random_df['id']:
    nutrition_url = f"https://api.spoonacular.com/recipes/{id}/nutritionWidget.json"
    nutrition_params = {'apiKey': spoonacular_key}

    nutrition_response = requests.get(nutrition_url, nutrition_params).json()
    nutrition_info.append(nutrition_response)

nutrition_df = pd.DataFrame(nutrition_info)
nutrition_df.to_csv(f'Resources/02_raw_data/info_master_{raw_folder()}.csv')

In [ ]:
# Parse results
metadata = pd.read_csv(f'Resources/01_recipe_IDs/initial_recipes_{recipe_folder()-1}.csv')
metadata

# Metadata
num_ingredients = []
carbs = []
# for row in metadata['extendedIngredients']:
#     print(row)

# Calories
calories_amt = []
calories_unit = []
calories_pct = []

# Saturated Fat
satfat_amt = []
satfat_unit = []
satfat_pct = []

# Sugar
sugar_amt = []
sugar_unit = []
sugar_pct = []

# Sodium
sodium_amt = []
sodium_unit = []
sodium_pct = []

# Protein
protein_amt = []
protein_unit = []
protein_pct = []

for idx, row in enumerate(nutrition_info):
    try:
        # recipe_id.append(nutrition_info[idx]['id'])
        # recipe_name.append(nutrition_info[idx]['title'])
        # recipe_likes.append(nutrition_info[idx]['aggregateLikes'])
        # meal_type.append(nutrition_info[idx]['dishTypes'])
        # cuisines.append(nutrition_info[idx]['cuisines'])
        # num_ingredients.append(len(recipe_info[idx]['extendedIngredients']))
        # ww_points.append(recipe_info[idx]['weightWatcherSmartPoints'])

        num_ingredients.append(len(nutrition_info[idx]['ingredients']))
        carbs.append(nutrition_info[idx]['carbs'])
        calories_amt.append(nutrition_info[idx]['nutrients'][0]['amount'])
        calories_unit.append(nutrition_info[idx]['nutrients'][0]['unit'])
        calories_pct.append(nutrition_info[idx]['nutrients'][0]['percentOfDailyNeeds'])
        
        satfat_amt.append(nutrition_info[idx]['nutrients'][2]['amount'])
        satfat_unit.append(nutrition_info[idx]['nutrients'][2]['unit'])
        satfat_pct.append(nutrition_info[idx]['nutrients'][2]['percentOfDailyNeeds'])
        
        sugar_amt.append(nutrition_info[idx]['nutrients'][5]['amount'])
        sugar_unit.append(nutrition_info[idx]['nutrients'][5]['unit'])
        sugar_pct.append(nutrition_info[idx]['nutrients'][5]['percentOfDailyNeeds'])
        
        sodium_amt.append(nutrition_info[idx]['nutrients'][7]['amount'])
        sodium_unit.append(nutrition_info[idx]['nutrients'][7]['unit'])
        sodium_pct.append(nutrition_info[idx]['nutrients'][7]['percentOfDailyNeeds'])
        
        protein_amt.append(nutrition_info[idx]['nutrients'][8]['amount'])
        protein_unit.append(nutrition_info[idx]['nutrients'][8]['unit'])
        protein_pct.append(nutrition_info[idx]['nutrients'][8]['percentOfDailyNeeds'])
    
    except:
        print(idx)
        pass

spoonacular_df = pd.DataFrame({
    'ID': metadata['id'],
    'Name': metadata['title'],
    'Likes': metadata['aggregateLikes'],
    'Meal Type': metadata['dishTypes'],
    'Cuisines': metadata['cuisines'],
    'N_ingredients': num_ingredients,
    'WW Smart Points': metadata['weightWatcherSmartPoints'],
    'Calories (Amount)': calories_amt,
    'Calories (Unit)': calories_unit,
    'Calories (% of Daily Needs)': calories_pct,
    'Saturated Fat (Amount)': satfat_amt,
    'Saturated Fat (Unit)': satfat_unit,
    'Saturated Fat (% of Daily Needs)': satfat_pct,
    'Sugar (Amount)': sugar_amt,
    'Sugar (Unit)': sugar_unit,
    'Sugar (% of Daily Needs)': sugar_pct,
    'Sodium (Amount)': sodium_amt,
    'Sodium (Unit)': sodium_unit,
    'Sodium (% of Daily Needs)': sodium_pct,
    'Protein (Amount)': protein_amt,
    'Protein (Unit)': protein_unit,
    'Protein (% of Daily Needs)': protein_pct,
    'Carbohydrates': carbs})

spoonacular_df.to_csv(f"Resources/03_simplified_data/initial_nutrition_{simple_folder()}.csv", index=False)

# Data Cleaning

In [18]:
simplified_data = pd.read_csv('Resources/03_simplified_data/initial_nutrition_0.csv')
simplified_data.columns

Index(['ID', 'Name', 'Likes', 'Meal Type', 'Cuisines', 'N_ingredients',
       'Calories (Amount)', 'Calories (Unit)', 'Calories (% of Daily Needs)',
       'Saturated Fat (Amount)', 'Saturated Fat (Unit)',
       'Saturated Fat (% of Daily Needs)', 'Sugar (Amount)', 'Sugar (Unit)',
       'Sugar (% of Daily Needs)', 'Sodium (Amount)', 'Sodium (Unit)',
       'Sodium (% of Daily Needs)', 'Protein (Amount)', 'Protein (Unit)',
       'Protein (% of Daily Needs)'],
      dtype='object')

In [20]:
raw_data = pd.read_csv('Resources/02_raw_data/info_master_0.csv')
raw_data['weightWatcherSmartPoints']

0     12
1      4
2     15
3     11
4      2
      ..
95     6
96     7
97    15
98    14
99    12
Name: weightWatcherSmartPoints, Length: 100, dtype: int64

In [21]:
test_df = simplified_data.insert(6, col='WW Smart Points', value=raw_data['weightWatcherSmartPoints'])
test_df

TypeError: DataFrame.insert() missing 1 required positional argument: 'value'